In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
import warnings
from pipelines.processing.processing_steps.data_augmentation import DataAugmentation
warnings.filterwarnings("ignore")

In [ ]:
df_trees_raw = pd.read_csv("../../data/raw/raw_trainset.csv")
df_trees = df_trees_raw.copy()
df_trees["time"] = pd.to_datetime(df_trees["time"])
df_aug_all = pd.read_csv("result.csv")
df_aug_all["time"] = pd.to_datetime(df_aug_all["time"])

In [ ]:
df_trees = df_trees.sort_values(['id', 'time'])
df_trees['time_diff'] = df_trees.groupby('id')['time'].diff()
mean_time_diff = df_trees.groupby('species')['time_diff'].mean()

print("AVG. DAYS PER SPECIES", mean_time_diff)
df_trees.drop(columns="time_diff")

In [ ]:
filter = df_trees.groupby("id").count().sort_values("time", ascending=True)
filter[filter["time"] == 100]

#### Goal:

#### - All species should be equaly represented
#### - All datapoints should be 2 week from one another appart --> 152 Datapoints per ID

In [ ]:
days = df_trees.time.max() - df_trees.time.min()
print(f"Ziel: {days/14} Datenpunkte pro Zeitreihe")

In [ ]:
px.histogram(df_trees[df_trees["species"] == "Norway_spruce"].groupby("id").count(), x="species")

In [ ]:
px.line(df_trees[df_trees["id"]==15037], x="time", y="b6")

In [ ]:
px.line(df_trees[df_trees["id"]==21027], x="time", y="b6")

In [ ]:
px.line(df_trees[df_trees["id"]==11], x="time", y="b6")

In [ ]:
example_id = 8999

In [ ]:
df_trees["year"] = df_trees["time"].dt.year
df_aug_all["year"] = df_aug_all["time"].dt.year

def compare_species_year(year, id, band="b12"):

    df_orig = df_trees[
        (df_trees["year"] == year) & (df_trees["id"] == id)
    ].sort_values("time")

    df_aug = df_aug_all[
        (df_aug_all["year"] == year) & (df_aug_all["id"] == id)
    ].sort_values("time")



    # Plot
    plt.figure(figsize=(10, 5))
    sns.lineplot(x="time", y=band, data=df_orig, label="Original", linewidth=2)
    sns.lineplot(x="time", y=band, data=df_aug, label="Augmented", linewidth=2, linestyle="--")

    plt.title(f"Example: {id} – {band} – Year {year}")
    plt.xlabel("Zeit")
    plt.ylabel("Wert")
    plt.legend()
    plt.tight_layout()
    plt.show()

compare_species_year(2019, example_id,"b12")

In [ ]:
def compare_species_year(id, band="b12"):

    df_orig = df_trees[df_trees["id"] == id].sort_values("time")
    df_aug = df_aug_all[df_aug_all["id"] == id].sort_values("time")

    plt.figure(figsize=(10, 5))
    sns.lineplot(x="time", y=band, data=df_orig, label="Original", linewidth=2)
    sns.lineplot(x="time", y=band, data=df_aug, label="Augmented", linewidth=2, linestyle=":")
    plt.scatter(df_orig["time"], df_orig[band], color="blue", s=40, alpha=0.6)
    plt.scatter(df_aug["time"], df_aug[band], color="orange", s=40, alpha=0.6)

    plt.title(f"Example: {id} – {band}")
    plt.xlabel("Zeit")
    plt.ylabel("Wert")
    plt.legend()
    plt.tight_layout()
    plt.show()

compare_species_year(example_id,"b12")

In [ ]:
print(f"Old dataframe lenght for ID {example_id}: {len(df_trees[df_trees["id"]==example_id])}")
print(f"New dataframe lenght for ID {example_id}: {len(df_aug_all[df_aug_all["id"]==example_id])}")

In [ ]:
#px.histogram(df_trees, x="species", title="New species distribution")

In [ ]:
#px.histogram(df_aug_all, x="species", title="New species distribution")

In [ ]:
filter = df_trees.groupby("id").size()
ids_le_max = filter[filter >= 152].index
df_ids_max = df_trees[df_trees["id"].isin(ids_le_max)].copy()

In [ ]:
df_trees

In [ ]:
df_aug_all[df_aug_all["id"]=="10536_aug"].sort_values(by="time")

In [ ]:
df_not_in_trees = df_aug_all[~df_aug_all.index.isin(df_trees.index)]
df_not_in_trees

In [ ]:
px.histogram(df_aug_all, x="species", title="New species distribution")